In [1]:
import numpy as np
from collections import namedtuple
import cv2
from pathlib import Path
import argparse
import os
from openvino.inference_engine import IENetwork, IECore
import csv
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from tensorflow import keras

c:\Users\de31\anaconda3\envs\lp\lib\site-packages\tensorflow\python\framework\dtypes.py:205: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  np.bool8: (False, True),
c:\Users\de31\anaconda3\envs\lp\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:326: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  np.bool8: (False, True),


## 전처리 먼저

In [2]:
folder_path_1 = "./data/구매(구매)"
folder_path_2 = "./data/구매(반품)"
folder_path_3 = "./data/구매(비교)"
folder_path_4 = "./data/구매(선택)"
folder_path_5 = "./data/구매(시험)"

In [3]:
def concat(path):
    file_lst = os.listdir(path)
    df_list = []
    for file in file_lst:
        file_name = os.path.join(path, file)
        df = pd.read_csv(file_name)
        df = df[df.NUMOFBODIES != 0].reset_index(drop=True)
        df['index_num'] = df.index
        df_list.append(df)
    #concat_df = pd.concat(df_list, ignore_index=True)
    return df_list

In [4]:
buy_mer = concat(folder_path_1)
refund_mer = concat(folder_path_2)
compar_mer = concat(folder_path_3)
select_mer = concat(folder_path_4)
test_mer = concat(folder_path_5)

In [5]:
mer_lst = [buy_mer, refund_mer, compar_mer, select_mer, test_mer]

In [6]:
def lengs(name):
    lengths = []
    for i in range(len(buy_mer)):
        lengths.append(len(buy_mer[i]))
    return lengths

In [7]:
lengsed = []
for i in mer_lst:
    lengsed.append(max(lengs(i)))
lengsed

[516, 516, 516, 516, 516]

In [8]:
buy_mer = pd.concat(buy_mer, ignore_index=True)
refund_mer = pd.concat(refund_mer, ignore_index=True)
compar_mer = pd.concat(compar_mer, ignore_index=True)
select_mer = pd.concat(select_mer, ignore_index=True)
test_mer = pd.concat(test_mer, ignore_index=True)

In [9]:
buy_mer['LABEL'] = 0  
refund_mer['LABEL'] = 1  
compar_mer['LABEL'] = 2
select_mer['LABEL'] = 3
test_mer['LABEL'] = 4

In [10]:
category_lst = [buy_mer, refund_mer, compar_mer, select_mer, test_mer]
full_data = pd.concat(category_lst, ignore_index=True)
full_data.shape

(216342, 57)

In [11]:
full_data

,NUMOFBODIES,ID,PELVIS_X,PELVIS_Y,PELVIS_CONFIDENCE_LEVEL,SPINE_NAVAL_X,SPINE_NAVAL_Y,SPINE_NAVA_CONFIDENCE_LEVEL,SPINE_CHEST_X,SPINE_CHEST_Y,...,ANKLE_RIGHT_X,ANKLE_RIGHT_Y,ANKLE_RIGHT_CONFIDENCE_LEVEL,HEAD_X,HEAD_Y,HEAD_CONFIDENCE_LEVEL,TIMESTAMP,FRAME_NUM,index_num,LABEL
0,1,0,346.55383,640.09490,2,316.11750,576.43010,2,280.24127,522.70620,...,434.75787,857.02515,2,239.81610,388.57196,2,14:46:38:326,63,0,0
1,1,0,374.96075,649.16760,2,356.46484,581.53500,2,337.61346,522.35820,...,387.14087,889.73960,2,265.55145,386.76953,2,14:46:38:435,66,1,0
2,1,0,429.70477,641.58417,2,411.60730,574.76430,2,388.89648,514.81540,...,392.70930,887.81946,2,331.37190,371.98608,2,14:46:38:518,69,2,0
3,1,0,495.43500,632.74255,2,477.77640,565.54290,2,457.34350,506.20636,...,410.20886,890.77374,2,402.76740,364.13617,2,14:46:38:630,72,3,0
4,1,0,547.60315,616.55760,2,527.22210,550.83923,2,507.37710,493.02690,...,479.45310,865.62000,2,463.35953,355.15625,2,14:46:38:726,75,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216337,1,0,1219.71860,662.44730,2,1230.92520,595.02515,2,1241.39250,538.59045,...,1134.70500,834.97217,1,1259.37120,403.96295,2,14:27:45:651,1677,405,4
216338,1,0,1271.88180,657.77160,2,1284.57190,588.35345,2,1297.53440,530.39655,...,1153.70860,863.37646,1,1314.07570,397.72302,2,14:27:45:748,1680,406,4
216339,1,0,1323.66500,661.49490,2,1337.05540,592.91000,2,1345.43380,533.05840,...,1218.74740,910.22986,1,1374.46680,396.15400,2,14:27:45:849,1683,407,4
216340,1,0,1368.66480,671.92664,2,1383.66700,600.05210,2,1394.43500,538.26140,...,1271.17430,939.52026,1,1428.83910,403.51175,2,14:27:45:945,1686,408,4


In [12]:
# 데이터와 레이블을 추출합니다.
data_without_label = full_data.drop(['ID','LABEL', 'TIMESTAMP', 'FRAME_NUM','PELVIS_CONFIDENCE_LEVEL', 'SPINE_NAVA_CONFIDENCE_LEVEL', 'SPINE_CHEST_CONFIDENCE_LEVEL', 'NECK_CONFIDENCE_LEVEL', 
                                     'SHOULDER_LEFT_CONFIDENCE_LEVEL', 'ELBOW_LEFT_CONFIDENCE_LEVEL', 'WRIST_LEFT_CONFIDENCE_LEVEL', 'SHOULDER_RIGHT_CONFIDENCE_LEVEL', 
                                     'ELBOW_RIGHT_CONFIDENCE_LEVEL', 'WRIST_RIGHT_CONFIDENCE_LEVEL', 'HIP_LEFT_CONFIDENCE_LEVEL', 'KNEE_LEFT_CONFIDENCE_LEVEL', 'ANKLE_LEFT_CONFIDENCE_LEVEL',
                                     'HIP_RIGHT_CONFIDENCE_LEVEL', 'KNEE_RIGHT_CONFIDENCE_LEVEL', 'ANKLE_RIGHT_CONFIDENCE_LEVEL', 'HEAD_CONFIDENCE_LEVEL'], axis=1).values
label_data = full_data['LABEL'].values

In [13]:
np.unique(label_data, return_counts=True)

(array([0, 1, 2, 3, 4], dtype=int64),
 array([43326, 42404, 44687, 43122, 42803], dtype=int64))

In [14]:
np.unique(data_without_label, return_counts=True)

(array([-111.89874, -105.88568,  -94.12323, ..., 2028.04   , 2040.5305 ,
        2073.094  ]),
 array([1, 1, 1, ..., 1, 1, 1], dtype=int64))

In [16]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

# 1. Data Splitting
# 데이터 분할
X_train, X_temp, y_train, y_temp = train_test_split(data_without_label, label_data, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.3, random_state=42)

In [17]:
print(X_train.dtype, y_train.dtype)  # 현재의 데이터 타입 확인

float64 int64


In [31]:
X_train.max()

2073.094

In [34]:
import tensorflow as tf

vocab_size = 2074  # This should be the maximum integer value in X_train plus one.
embedding_dimension = 100
sequence_length = 516  # Or the correct length of your sequences.

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dimension, input_length=sequence_length),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(5, activation='softmax')  # 5 classes
])

ValueError: Input 0 of layer "lstm_13" is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (None, 64)

In [27]:

# 3. Model Compilation
# 모델 컴파일
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model_file_path = "./model/best_lstm_model.h5"
checkpoint_cb = keras.callbacks.ModelCheckpoint(model_file_path)
# - 자동 훈련 멈추기 함수 사용 : 추가 훈련 epoch 2회, 가중치 업데이트
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10,
                                        restore_best_weights=True)


# 4. Model Training
# 모델 훈련
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=1000,
                    batch_size=32, callbacks=[checkpoint_cb, early_stopping_cb])

# 5. Model Evaluation
# 모델 평가
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy*100:.2f}%")


Epoch 1/1000


ValueError: in user code:

    File "c:\Users\de31\anaconda3\envs\lp\lib\site-packages\keras\engine\training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\de31\anaconda3\envs\lp\lib\site-packages\keras\engine\training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\de31\anaconda3\envs\lp\lib\site-packages\keras\engine\training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\de31\anaconda3\envs\lp\lib\site-packages\keras\engine\training.py", line 859, in train_step
        y_pred = self(x, training=True)
    File "c:\Users\de31\anaconda3\envs\lp\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\de31\anaconda3\envs\lp\lib\site-packages\keras\engine\input_spec.py", line 264, in assert_input_compatibility
        ra

In [ ]:
model.summary()

In [7]:
age = {1:{2: "도도둑"}}

In [8]:
age[1][2] = 'eh' 

{1: {2: 'eh'}}